In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Glucocorticoid_Sensitivity/GSE33649'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Inter-ethnic differences in lymphocyte  sensitivity to glucocorticoids reflect variation in transcriptional response"
!Series_summary	"Glucocorticoids (GCs) are steroid hormones widely  used as pharmaceutical interventions, which act mainly by regulating gene expression levels. A large fraction of patients (~30%), especially those of African descent, show a weak response to treatment. To interrogate the contribution of variable transcriptional response to inter-ethnic differences, we measured  in vitro lymphocyte GC sensitivity (LGS) and transcriptome-wide response to GCs in peripheral blood mononuclear cells (PBMCs) from African-American and European-American healthy donors. We found that transcriptional response after 8hrs treatment was significantly correlated with variation in LGS within and  between populations. We found that NFKB1, a gene previously found to predict LGS within populations, was more strongly downregulated in European-American

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Initialize the sample_characteristics dictionary
sample_characteristics = {
    0: ['cell type: peripheral blood mononuclear cells'],
    1: ['population: African-American', 'population: European-American'],
    2: ['treatment: dexamethasone', 'treatment: vehicle (EtOH)'],
    3: ['in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 89.43486',
        'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 95.88507',
        'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 95.22036',
        'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 92.86704',
        'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 93.71633',
        'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 96.76962',
        'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 88.55031',
        'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 90.09957',
        'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 94.17097',
        'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 86.97089',
        'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 98.34904',
        'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 91.14896'],
    4: ['duration of treatment (hours): 8', 'duration of treatment (hours): 24'],
    5: ['gender: female', 'gender: male'],
    6: ['age (years): 44.15342', 'age (years): 24.72329', 'age (years): 32.37808',
        'age (years): 20.38082', 'age (years): 21.2411', 'age (years): 22.54247',
        'age (years): 26.13973', 'age (years): 21.5616', 'age (years): 21.9863',
        'age (years): 26.76712', 'age (years): 23.59452', 'age (years): 23.47945']
}

# Determine if gene expression data is available
is_gene_available = True

# Identify the keys for the required variables if available
trait_row = 3 if len({x.split(": ")[1] for x in sample_characteristics[3]}) > 1 else None
age_row = 6 if len({x.split(": ")[1] for x in sample_characteristics[6]}) > 1 else None
gender_row = 5 if len({x.split(": ")[1] for x in sample_characteristics[5]}) > 1 else None

# Trait data type conversion function
def convert_trait(value):
    try:
        value = value.split(": ")[1]
        return float(value) if value else None
    except:
        return None

# Age data type conversion function
def convert_age(value):
    try:
        value = value.split(": ")[1]
        return float(value) if value else None
    except:
        return None

# Gender data type conversion function
def convert_gender(value):
    try:
        value = value.split(": ")[1]
        if value.lower() == 'female':
            return 0
        elif value.lower() == 'male':
            return 1
    except:
        return None
    return None

# Save cohort information
save_cohort_info('GSE33649', './preprocessed/Glucocorticoid_Sensitivity/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction (if clinical data is available)
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Glucocorticoid_Sensitivity', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Glucocorticoid_Sensitivity/trait_data/GSE33649.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM832137': [89.43486, 44.15342, 0.0], 'GSM832138': [89.43486, 44.15342, 0.0], 'GSM832139': [89.43486, 44.15342, 0.0], 'GSM832140': [89.43486, 44.15342, 0.0], 'GSM832141': [95.88507, 24.72329, 0.0], 'GSM832142': [95.88507, 24.72329, 0.0], 'GSM832143': [95.88507, 24.72329, 0.0], 'GSM832144': [95.88507, 24.72329, 0.0], 'GSM832145': [95.22036, 32.37808, 1.0], 'GSM832146': [95.22036, 32.37808, 1.0], 'GSM832147': [95.22036, 32.37808, 1.0], 'GSM832148': [95.22036, 32.37808, 1.0], 'GSM832149': [92.86704, 20.38082, 0.0], 'GSM832150': [92.86704, 20.38082, 0.0], 'GSM832151': [92.86704, 20.38082, 0.0], 'GSM832152': [92.86704, 20.38082, 0.0], 'GSM832153': [93.71633, 21.2411, 0.0], 'GSM832154': [93.71633, 21.2411, 0.0], 'GSM832155': [93.71633, 21.2411, 0.0], 'GSM832156': [93.71633, 21.2411, 0.0], 'GSM832157': [96.76962, 22.54247, 0.0], 'GSM832158': [96.76962, 22.54247, 0.0], 'GSM832159': [96.76962, 22.54247, 0.0], 'GSM832160': [96.76962, 22.54247, 0.0], 'GSM832161': [88.55031, 26.13973, 1.0], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate keys for mapping
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Glucocorticoid_Sensitivity/gene_data/GSE33649.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Glucocorticoid_Sensitivity')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE33649', './preprocessed/Glucocorticoid_Sensitivity/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Glucocorticoid_Sensitivity/GSE33649.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Glucocorticoid_Sensitivity':
  25%: 89.9333925
  50% (Median): 93.291685
  75%: 95.3865375
Min: 86.97089
Max: 98.34904
The distribution of the feature 'Glucocorticoid_Sensitivity' in this dataset is fine.

Quartiles for 'Age':
  25%: 21.880125
  50% (Median): 23.536985
  75%: 26.296577499999998
Min: 20.38082
Max: 44.15342
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 24 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

